# Behavior Clone Model

---
## Loading Data

In [1]:
import csv
import cv2

lines = [] 
with open('../driving-data/data/driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        if line[0] != 'center':
            lines.append(line)

images = []
measurements = []
for line in lines:
    for i in range(3):
        source_path = line[i]
        current_path = '../driving-data/data/' + source_path.strip()
        image = cv2.imread(current_path)
        images.append(image)
        measurement = float(line[3])
        correction = 0.2 # this is a parameter to tune
        steering_left = measurement + correction
        steering_right = measurement - correction
        if i == 1:
            measurement = steering_left
        elif i == 2:
            measurement = steering_right
        measurements.append(measurement)
print ('training data ', len(images), len(measurements))

training data  24108 24108


## Data Process

In [2]:
augmented_images, augmented_measurements = [], []
for image,measurement in zip(images, measurements):
    augmented_images.append(image)
    augmented_measurements.append(measurement)
    augmented_images.append(cv2.flip(image,1))
    augmented_measurements.append(measurement*-1.0)
    
print ('training data ', len(augmented_images), len(augmented_measurements))

training data  48216 48216


In [3]:
import numpy as np

x_train = np.array(augmented_images)
y_train = np.array(augmented_measurements)

from keras.models import Sequential
from keras.layers import Flatten, Dense, Lambda, Convolution2D
from keras.layers import Cropping2D

model = Sequential()
model.add(Lambda(lambda x: x/255.0 - 0.5, input_shape=(160,320,3)))
model.add(Cropping2D(cropping=((70,25), (0,0))))
model.add(Convolution2D(24,5,5,subsample=(2,2),activation='relu'))
model.add(Convolution2D(36,5,5,subsample=(2,2),activation='relu'))
model.add(Convolution2D(48,5,5,subsample=(2,2),activation='relu'))

model.add(Convolution2D(64,3,3, activation='relu'))
model.add(Convolution2D(64,3,3, activation='relu'))

model.add(Flatten())
model.add(Dense(100))
model.add(Dense(50))
model.add(Dense(10))

model.add(Dense(1))

model.compile(loss='mse' ,optimizer='adam')
model.fit(x_train, y_train, validation_split=0.2, shuffle=True, epochs=5)

model.save('model.h5')

KeyboardInterrupt: 